In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gopal/articles.csv


In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pickle

# Load the dataset
data = pd.read_csv("/kaggle/input/gopal/articles.csv")



In [3]:
# Convert to Parquet format
data.to_parquet("articles.parquet", index=False)

In [4]:
# Load the dataset from Parquet
data = pd.read_parquet("your_dataset.parquet")


# Combine information from all relevant columns into a single text column
text_data = data.apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)


In [13]:
# Text Processing
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)  # Limiting features
tfidf_matrix = tfidf_vectorizer.fit_transform(text_data)



In [13]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import pickle
import spacy

# Load the English language model from spaCy
nlp = spacy.load("en_core_web_sm")

# Load the dataset
data = pd.read_parquet("articles.parquet")

# Impute missing values if any
data.fillna("", inplace=True)

# Combine information from all relevant columns into a single text column
text_data = data.apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)

# Text Processing using spaCy
processed_text_data = []
for doc in nlp.pipe(text_data, disable=["parser", "ner"]):
    tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    processed_text_data.append(" ".join(tokens))

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(processed_text_data)

# Splitting Data
X_train, X_test = train_test_split(tfidf_matrix, test_size=0.2, random_state=42)

# Training the Model using k-nearest neighbors
# You can adjust the number of neighbors (n_neighbors) as needed
knn_model = NearestNeighbors(n_neighbors=5, algorithm='auto')
knn_model.fit(X_train)

# Saving the Model
with open("recommender_knn_model_2.pkl", "wb") as f:
    pickle.dump(knn_model, f)

# Testing the Model
# Load the model
with open("recommender_knn_model_2.pkl", "rb") as f:
    loaded_knn_model = pickle.load(f)

# Example Usage
# Assume 'new_description' is the new description you want to find recommendations for
new_description = "Your new input description"
doc = nlp(new_description)
processed_new_description = " ".join([token.lemma_ for token in doc if not token.is_stop and token.is_alpha])
new_description_tfidf = tfidf_vectorizer.transform([processed_new_description])

# Find the indices of the nearest neighbors
distances, indices = loaded_knn_model.kneighbors(new_description_tfidf)

# Get the indices of the most similar products
similar_indices = indices[0]

# Get the recommended products
recommended_products = data.iloc[similar_indices]
print(recommended_products)


       article_id  product_code        prod_name  product_type_no  \
52274   700946004        700946   LEO HOOD 14.99              308   
53146   703296007        703296      Nick Puffer              262   
1868    379780048        379780  Rita Ringer Tee              255   
49575   694923001        694923    LEGGINGS MEOW              272   
38      141661025        141661    Velour pyjama              295   

              product_type_name  product_group_name  graphical_appearance_no  \
52274                    Hoodie  Garment Upper body                  1010014   
53146                    Jacket  Garment Upper body                  1010016   
1868                    T-shirt  Garment Upper body                  1010017   
49575                  Trousers  Garment Lower body                  1010023   
38     Pyjama jumpsuit/playsuit           Nightwear                  1010002   

      graphical_appearance_name  colour_group_code colour_group_name  ...  \
52274           Placement p

In [16]:

# Assume 'new_description' is the new description you want to find recommendations for
new_description = "Sweat Shirt"
doc = nlp(new_description)
processed_new_description = " ".join([token.lemma_ for token in doc if not token.is_stop and token.is_alpha])
new_description_tfidf = tfidf_vectorizer.transform([processed_new_description])

# Find the indices of the nearest neighbors
distances, indices = loaded_knn_model.kneighbors(new_description_tfidf)

# Get the indices of the most similar products
similar_indices = indices[0]

# Get the recommended products
recommended_products = data.iloc[similar_indices]
print(recommended_products)

       article_id  product_code                   prod_name  product_type_no  \
83511   812351001        812351               Aria Cardigan              252   
46590   686242004        686242  Pat ls double pocket plaid              259   
48809   693261002        693261              Anthea deal PJ              297   
16369   569976009        569976                MASON C-NECK              252   
38527   657902002        657902              LOGG Tyra hood              254   

      product_type_name  product_group_name  graphical_appearance_no  \
83511           Sweater  Garment Upper body                  1010010   
46590             Shirt  Garment Upper body                  1010004   
48809        Pyjama set           Nightwear                  1010008   
16369           Sweater  Garment Upper body                  1010016   
38527               Top  Garment Upper body                  1010010   

      graphical_appearance_name  colour_group_code colour_group_name  ...  \
83511    

In [17]:
data.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
